<a href="https://colab.research.google.com/github/DamianKocemba/Split-Explore-Merge-Pdf-File/blob/main/ZawiadomieniaWypisy_KW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# instalacja biblioteki
#pip install PyPDF2

In [6]:
from PyPDF2 import PdfFileReader, PdfFileWriter, PdfFileMerger
import re
import os

In [7]:
# definicja zmiennych (sciezek i nazw plikow)
obreb = "ZERKOW"

path = "/content/ZAWIADOMIENIA_DO_KW/"
_zawiadomienia = 'Zawiadomienia_{}_KW.pdf'.format(obreb)
_wypisy = "Wypisy_{}.pdf".format(obreb)

In [8]:
# utworzenie tymczasowych folderow
if not os.path.exists(path):
    os.mkdir(path)

if not os.path.exists(path+"zawiadomienia_temp"):
    os.mkdir(path+"zawiadomienia_temp")

if not os.path.exists(path+"wypisy_temp"):
    os.mkdir(path+"wypisy_temp")

In [9]:
# Teraz nalezy zaimportować pliki pdf z Zawiadomieniami i Wypisami

In [ ]:
# ekstrakcja zawiadomien z pliku .pdf (po numerze jednostki rejestrowej)

pdf_zaw = PdfFileReader(path+_zawiadomienia)
key_text = 'Jednostka rejestrowa'

zawiadomienia = {}
for page in range(pdf_zaw.numPages):
    PgObj_zaw = pdf_zaw.getPage(page)
    Text_zaw = PgObj_zaw.extractText()
    if re.search(key_text, Text_zaw):
        print("nr strony: ", page+1)
        jedn_rej = re.findall('[0-9]+ - Gruntowa', Text_zaw)
        print("Jednostka rej.: ",jedn_rej[0].split()[0])
        zawiadomienia[int(page)] =  "zaw_G_" + jedn_rej[0].split()[0]


lista_stron_zaw = [s for s in zawiadomienia.keys()]

for i in range(len(lista_stron_zaw)):
    try:
        output = PdfFileWriter()
        for page_zaw in range(lista_stron_zaw[i],lista_stron_zaw[i+1]):
            output.addPage(pdf_zaw.getPage(page_zaw))
    except IndexError:
        output = PdfFileWriter()
        for page_zaw in range(lista_stron_zaw[i],pdf_zaw.getNumPages()):
            output.addPage(pdf_zaw.getPage(page_zaw))
    
    with open(path+"zawiadomienia_temp/"+'{}.pdf'.format(zawiadomienia[lista_stron_zaw[i]]), 'wb') as f:
            output.write(f)




In [ ]:
# ekstrakcja wypisow z pliku .pdf (po numerze jednostki rejestrowej)

pdf_wyp = PdfFileReader(path+_wypisy)

wypisy = {}
for page in range(pdf_wyp.numPages):
    PgObj_wyp = pdf_wyp.getPage(page)
    Text_wyp = PgObj_wyp.extractText()
    if re.search(key_text, Text_wyp):
        print("Nr strony: ",page+1)
        jedn_rej_wyp = re.findall("G.[0-9]+", Text_wyp)
        print("Jednostka_rej.: ", jedn_rej_wyp)
        wypisy[int(page)] = "wyp_" + jedn_rej_wyp[0].replace(".", "_")


lista_stron_wyp = [z for z in wypisy.keys()]

for j in range(len(lista_stron_wyp)):
    try:
        output_wyp = PdfFileWriter()
        for page_wyp in range(lista_stron_wyp[j],lista_stron_wyp[j+1]):
            output_wyp.addPage(pdf_wyp.getPage(page_wyp))
    except IndexError:
        output_wyp = PdfFileWriter()
        for page_wyp in range(lista_stron_wyp[j],pdf_wyp.getNumPages()):
            output_wyp.addPage(pdf_wyp.getPage(page_wyp))
    
    with open(path+"wypisy_temp/"+"{}.pdf".format(wypisy[lista_stron_wyp[j]]), 'wb') as f2:
        output_wyp.write(f2)


In [12]:
# utworzenie list zawiadomien i wypisow

lista_zawiadomien = [z.replace("zaw_","") for z in os.listdir(path+"/zawiadomienia_temp")]

lista_wypisow = os.listdir(path+"/wypisy_temp")
lista_wypisow = sorted(lista_wypisow, key = lambda x: (int(re.sub('\D','',x)),x))


In [ ]:
# polaczenie plikow w jeden

merger = PdfFileMerger()

for pdf_file_wyp in lista_wypisow:
    #print(pdf_file_wyp)
    print("*"*30)
    wypis = pdf_file_wyp.replace("wyp_","")
    print("Wypis: ", wypis)
    for pdf_file_zaw in os.listdir(path+"/zawiadomienia_temp"):
        #print(pdf_file_zaw)
        zawiadomienie = pdf_file_zaw.replace("zaw_","")
        #print("Zawiadomienie: ", zawiadomienie)
        if wypis == zawiadomienie:
            merger.append(path+"zawiadomienia_temp/"+pdf_file_zaw)
            merger.append(path+"wypisy_temp/"+pdf_file_wyp)
        
    if wypis not in lista_zawiadomien:
        with open(path+"brakujace_zawiadomienia.txt","a") as f:
            f.write(wypis)

merger.write(path+"Zawiadomienia_Wypisy_{}.pdf".format(obreb))
merger.close

In [14]:
# usuniecie folderow tymczasowych

def delete_temp(directory):
    for file in os.listdir(directory):
        os.remove(directory+"/"+file)

    os.rmdir(directory)

delete_temp(path+"/zawiadomienia_temp")
delete_temp(path+"/wypisy_temp")


In [15]:
# Created by Damian Kocemba